In [ ]:

# === Configuración de origen de datos ===
import os, io, requests, pandas as pd

GH_USER   = "RobertoSam"
GH_REPO   = "Final_MetodosComputacionales"
GH_BRANCH = "main"
GH_DIR    = ""   # si mueves los CSV a /data, cámbialo a "data"

DATA_DIR_LOCAL = "/Users/robert/Downloads/evaluacinfinalcasos"
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN", None)

def github_raw_url(filename):
    return f"https://raw.githubusercontent.com/{GH_USER}/{GH_REPO}/{GH_BRANCH}/{GH_DIR}{filename}"

def read_csv_smart(filename, sep=",", **kwargs):
    url = github_raw_url(filename)
    try:
        headers = {}
        if GITHUB_TOKEN is not None:
            headers["Authorization"] = f"token {GITHUB_TOKEN}"
        r = requests.get(url, headers=headers, timeout=30)
        r.raise_for_status()
        return pd.read_csv(io.StringIO(r.text), sep=sep, **kwargs)
    except Exception as e:
        print(f"[Aviso] No se pudo leer desde GitHub: {url}\n -> {e}\nIntentando ruta local...")
    local_path = os.path.join(DATA_DIR_LOCAL, filename)
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"No se encontró el archivo en GitHub ni local. Intenté:\n- GitHub: {url}\n- Local: {local_path}")
    return pd.read_csv(local_path, sep=sep, **kwargs)


# Caso 3: Rendimiento Escolar
data = read_csv_smart("student-mat.csv", sep=";")
# ... pipeline de regresión, métricas y comparación ...
